# Project: Vehicle Counting with Line Crossing using OpenCV

This project implements a basic **vehicle detection and counting system** using Python and OpenCV.  
It uses **background subtraction** to detect moving vehicles and counts them when they **cross a virtual line** on the road.

---
## 🔍 Steps Breakdown

### 1. **Video Input & Frame Preprocessing**
Read frames from a video file using OpenCV’s `cv2.VideoCapture()`.  
Each frame is converted to **grayscale** and **blurred** to reduce noise and improve motion detection consistency.

---

### 2. **Background Subtraction**
Use `cv2.bgsegm.createBackgroundSubtractorMOG()` to detect **moving objects**.  
This isolates vehicles by subtracting the current frame from a background model, ideal for **static camera setups**.

---

### 3. **Morphological Operations**
Apply a series of **morphological transformations** (dilation and closing) to clean up the motion mask:
- Remove small noise and holes
- Merge fragmented regions
- Create smooth, solid blobs for easier contour detection

---

### 4. **Contour Detection**
Use `cv2.findContours()` to find outlines of moving objects in the cleaned mask.  
Each contour is a candidate for being a vehicle.

---

### 5. **Filtering Valid Vehicles**
Filter out small contours that don’t meet the **minimum width and height** thresholds.  
This helps ignore false positives like shadows or debris.

---

### 6. **Drawing Bounding Boxes & Center Points**
For each valid contour:
- Draw a **bounding box** around the detected vehicle
- Compute its **center point**
- Store the center in a list of detections for counting purposes

---

### 7. **Line Crossing Detection**
A horizontal line is drawn across the frame at a fixed position.  
If a vehicle’s center point **crosses this line** (within a margin of error), it is counted:
- The counter is incremented
- The point is removed from the detection list to prevent double-counting

---

### 8. **Displaying Results**
Use `cv2.putText()` to display the current **vehicle count** in real-time on the video feed.  
Also visualize:
- The detecon line
- Bounding boxes
- Vehicle center points

---

## ✅ Outcome

- Detects and tracks moving vehicles
- Counts vehicles as they cross a predefined line
- Displays visual feedback with bounding boxes anouffset < center[1] < line_position + offset:
        vehicle_count += 1
        detections.remove(center)

   Pdeo.mp4')
frame_delay = 60
ect detection (e.g., YOLO, SSD)
- GUI dashboard for live monitoring and analytics

---


In [11]:
import cv2
import numpy as np
from time import sleep

# Minimum width and height of detected vehicle
min_width = 80
min_height = 80

# Line position and offset for detection
line_position = 550
offset = 6

# Time delay between frames (based on FPS)
frame_delay = 60

# Variables to hold detected centers and vehicle count
detections = []
vehicle_count = 0

# Function to get the center of the rectangle (vehicle)
def get_center(x, y, w, h):
    cx = x + int(w / 2)
    cy = y + int(h / 2)
    return cx, cy

# Load video
cap = cv2.VideoCapture('video.mp4')

# Background subtractor
#background_subtractor = cv2.bgsegm.createBackgroundSubtractorMOG() # old algorithm
#background_subtractor = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=True) # new algorithms
background_subtractor = cv2.createBackgroundSubtractorKNN() # good with fast moving
while True:
    ret, frame = cap.read()
    if not ret:
        break  # Exit if video ends

    # Frame rate control
    sleep(1 / frame_delay)

    # Preprocessing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 5)
    mask = background_subtractor.apply(blurred)
    #dilated = cv2.dilate(mask, np.ones((5, 5))) # the orginal code
    kernel = np.ones((5, 5), np.uint8)
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # Morphological transformations to reduce noise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel) # here I changed dilated to opening
    closing = cv2.morphologyEx(closing, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Draw detection line
    cv2.line(frame, (25, line_position), (1200, line_position), (176, 130, 39), 2)

    # Analyze each contour
    for cnt in contours:
        (x, y, w, h) = cv2.boundingRect(cnt)
        if w >= min_width and h >= min_height:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            center = get_center(x, y, w, h)
            detections.append(center)
            cv2.circle(frame, center, 4, (0, 0, 255), -1)

    # Count vehicles crossing the line
    for (cx, cy) in detections:
        if (line_position - offset) < cy < (line_position + offset):
            vehicle_count += 1
            cv2.line(frame, (25, line_position), (1200, line_position), (0, 127, 255), 3)
            detections.remove((cx, cy))
            print(f"Vehicle count: {vehicle_count}")

    # Display count on screen
    cv2.putText(frame, f"VEHICLE COUNT : {vehicle_count}", (320, 70),
                cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255), 4)

    # Show video and mask
    cv2.imshow("Original Video", frame)
    cv2.imshow("Detection", closing)

    # Break loop on ESC key
    if cv2.waitKey(1) == 27:
        break

# Clean up
cap.release()
cv2.destroyAllWindows()


Vehicle count: 1
Vehicle count: 2
Vehicle count: 3
Vehicle count: 4
Vehicle count: 5
Vehicle count: 6
Vehicle count: 7
Vehicle count: 8
Vehicle count: 9
Vehicle count: 10
Vehicle count: 11
Vehicle count: 12
Vehicle count: 13
Vehicle count: 14
Vehicle count: 15
Vehicle count: 16
Vehicle count: 17
Vehicle count: 18
Vehicle count: 19
Vehicle count: 20
Vehicle count: 21
Vehicle count: 22
Vehicle count: 23
Vehicle count: 24
Vehicle count: 25
Vehicle count: 26
Vehicle count: 27
Vehicle count: 28
Vehicle count: 29
